# Dataset

### Import Dataset

In [1]:
import pandas as pd
import numpy as np

DATASET_PATH = 'data/Dataset-Mental-Health-with-Tags.xlsx'
datasets = pd.read_excel(DATASET_PATH)
answers_set = datasets[['Jawaban', 'Tag']]
datasets = datasets[['Pertanyaan', 'Tag']]
datasets.info()
tags = np.unique(datasets['Tag'])
print(datasets.head())
print(tags)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pertanyaan  500 non-null    object
 1   Tag         500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB
                                          Pertanyaan   Tag
0  Mengapa saya merasa semakin stres setelah berm...  tag1
1  Apa yang menyebabkan saya merasa depresi setel...  tag1
2  Kenapa perasaan saya memburuk setelah melihat ...  tag1
3  Mengapa saya merasa cemas setelah menghabiskan...  tag1
4  Apa alasan di balik perasaan tertekan saya set...  tag1
['tag1' 'tag10' 'tag11' 'tag12' 'tag13' 'tag14' 'tag15' 'tag16' 'tag17'
 'tag18' 'tag19' 'tag2' 'tag20' 'tag21' 'tag22' 'tag23' 'tag24' 'tag25'
 'tag3' 'tag4' 'tag5' 'tag6' 'tag7' 'tag8' 'tag9']


In [2]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import re

factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop = stopwords.words('indonesian')

def prepare_question(text: str) -> str:
  text = text.lower()
  text = re.sub(r'[,](?!\s)', ', ', text)  # Add spasi pada koma tanpa spasi
  text = re.sub(r'[.](?!\s)', '. ', text)  # Add spasi pada titik tanpa spasi
  text = text.replace('\\t', ' ').replace('\\n', ' ').replace('\\u', ' ')  # Hapus tab, new line, , dll
  text = text.encode('ascii', 'replace').decode('ascii')  # Hapus karakter non ASCII (emoticon, chinese word, dll)
  text = re.sub(r"(?i)(?:https?:\/\/)?(?:www\.)?(?:[a-zA-Z0-9-.]+)(?:\.[a-zA-Z]{2,6})(?:\/[^\s\r\n]*)?", "", text)  # Hapus URL
  text = re.sub(r'[\!\"\#\$\%\&\'\(\)\*\+\,\.\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)  # Ubah tanda baca ke spasi
  text = text.strip()  # Hapus whitespace di depan/belakang teks
  text = re.sub(r'\\s+', ' ', text)  # Hapus double++ spasi
  text = re.sub(r'\\s+(?=\.)', '', text)  # Hapus spasi sebelum titik
  text = re.sub(r'\.{2,}', r'\.', text)  # Hapus titik++
  text = ' '.join([word for word in text.split() if word not in (stop)])  # Hapus stopwords
  return stemmer.stem(text)  # Return hasil stemming
  # return text

In [3]:
new_datasets = datasets.sample(frac=1, random_state=42)
new_datasets['stem'] = new_datasets['Pertanyaan'].apply(prepare_question)
new_datasets.head()

new_datasets.to_excel('data/Dataset-Mental-Health-with-Tags-Preprocessed.xlsx', index=False)

### Encode dataset

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode labels
le = LabelEncoder()
new_datasets['label'] = le.fit_transform(new_datasets['Tag'])
# Check class distribution
print(new_datasets['label'].value_counts())

# Split dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    list(new_datasets['Pertanyaan']), list(new_datasets['label']), random_state=42, test_size=0.1
)

label
10    20
7     20
14    20
3     20
5     20
18    20
24    20
8     20
2     20
16    20
9     20
6     20
11    20
19    20
17    20
20    20
13    20
1     20
0     20
15    20
22    20
12    20
21    20
23    20
4     20
Name: count, dtype: int64


# Prepare Training

### Define models to train

In [5]:
from transformers import BertForSequenceClassification
from transformers import AlbertForSequenceClassification
from transformers import DistilBertForSequenceClassification
from transformers import MobileBertForSequenceClassification
from transformers import AutoTokenizer, BertTokenizer
from transformers import Trainer, TrainingArguments
import torch
import time

In [6]:
models = [
    ("bert-base-uncased", BertForSequenceClassification, "bert-base-uncased"),
    ("bert-base-multilingual-uncased", BertForSequenceClassification, "bert-base-multilingual-uncased"),
    ("albert/albert-base-v2", AlbertForSequenceClassification, "albert-base-v2"),
    ("distilbert-base-uncased", DistilBertForSequenceClassification, "distilbert-base-uncased"),
    ('google/mobilebert-uncased', MobileBertForSequenceClassification, "mobilebert-uncased"),
    ('cahya/bert-base-indonesian-522M', BertForSequenceClassification, "bert-base-indonesian-522M"),
]

training_durations = {}

### Check device used

In [7]:
# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Define PyTorch dataset

In [8]:
# Define PyTorch datasets
class PyTorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

### Create training function

In [9]:
def train_model(model_name, model_class, model_nickname):
    # Load tokenizer and model
    model = model_class.from_pretrained(model_name, num_labels=len(tags))
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenize training and validation texts
    train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
    val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

    # Create PyTorch datasets
    train_datasets = PyTorchDataset(train_encodings, train_labels)
    eval_datasets = PyTorchDataset(val_encodings, val_labels)

    # Prepare training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{model_name}",
        eval_strategy="epoch",
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=20,
        logging_dir=f"./logs/{model_name}",
    )

    # Define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_datasets,
        eval_dataset=eval_datasets,
    )

    # Fine-tune the model
    start_time = time.time()
    trainer.train()
    end_time = time.time()
    training_durations[model_nickname] = end_time - start_time
    print(f"Training time for {model_nickname}: {training_durations[model_nickname]} seconds")

    model.save_pretrained(f"./results/{model_name}_fine-tuned_mental-health")
    tokenizer.save_pretrained(f"./results/{model_name}")
    torch.cuda.empty_cache()


### Start training

In [10]:
for model_name, model_class, model_nickname in models:
    print(f"Training {model_name}")
    train_model(model_name, model_class, model_nickname)
    print("*" * 100)
    print()

for training_time in training_durations:
    print(f"Training time for {training_time}: {training_durations[training_time]} seconds")

Training bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/300 [00:00<?, ?it/s]

c:\Users\PC\Documents\Cristian\Paper-Q3\.conda\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/2 [00:00<?, ?it/s]

...

{'eval_loss': 0.10558206588029861, 'eval_runtime': 0.042, 'eval_samples_per_second': 1189.415, 'eval_steps_per_second': 47.577, 'epoch': 18.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.10759995132684708, 'eval_runtime': 0.042, 'eval_samples_per_second': 1189.408, 'eval_steps_per_second': 47.576, 'epoch': 19.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.10719986259937286, 'eval_runtime': 0.0245, 'eval_samples_per_second': 2038.782, 'eval_steps_per_second': 81.551, 'epoch': 20.0}
{'train_runtime': 23.2434, 'train_samples_per_second': 387.207, 'train_steps_per_second': 12.907, 'train_loss': 0.31978235880533856, 'epoch': 20.0}
Training time for bert-base-indonesian-522M: 23.38695478439331 seconds
****************************************************************************************************

Training time for bert-base-uncased: 36.680134534835815 seconds
Training time for bert-base-multilingual-uncased: 31.358631372451782 seconds
Training time for albert-base-v2: 36.128053426742554 seconds
Training time for distilbert-base-uncased: 20.19717836380005 seconds
Training time for mobilebert-uncased: 47.37594723701477 seconds
Training time for bert-base-indonesian-522M: 23.38695478439331 seconds


# Deploy Trained Models

In [11]:
import re
from torchinfo import summary

In [12]:
TEST_SET_PATH = 'data/Test-Set-Mental-Health-with-Tags.xlsx'
test_sets = pd.read_excel(TEST_SET_PATH)
test_sets = test_sets[['Pertanyaan', 'Tag']]
test_sets.info()

test_sets = test_sets.sample(frac=1, random_state=42)
# new_test_sets['stem'] = new_test_sets['Pertanyaan'].apply(prepare_question)
# print(new_test_sets.head())
# print(np.unique(new_test_sets['Tag']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Pertanyaan  250 non-null    object
 1   Tag         250 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [14]:
def get_prediction(text, tokenizer, model):
    cleaned_text = re.sub(r"[^a-zA-Z ]+", "", text)
    cleaned_text = prepare_question(cleaned_text)

    # Tokenize input text
    inputs = tokenizer(
        cleaned_text, padding=True, truncation=True, return_tensors="pt"
    ).to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted logits
    logits = outputs.logits

    # Convert logits to probabilities
    probs = torch.softmax(logits, dim=-1)

    # Get predicted label (index of the maximum probability)
    predicted_label_index = torch.argmax(probs, dim=-1).item()

    # Get corresponding label name
    predicted_label = tags[predicted_label_index]

    return predicted_label

In [19]:
def deploy_model(model_name, model_class):
    model = model_class.from_pretrained(f"./results/{model_name}_fine-tuned_mental-health", num_labels=len(tags))
    tokenizer = AutoTokenizer.from_pretrained(f"./results/{model_name}")

    model.to(device)
    print(summary(model))

    prediction_results = []
    prediction_time = []

    for index, row in test_sets.iterrows():
        start_time = time.time()
        text = row['Pertanyaan']
        predicted_label = get_prediction(text, tokenizer, model)
        end_time = time.time()

        print(f"Input: {text}")
        print(f"Tag: {row['Tag']}")
        print(f"Predicted: {predicted_label}")
        for i in range(len(answers_set)):
            if answers_set.iloc[i]['Tag'] == predicted_label:
                print(f"Answer: {answers_set.iloc[i]['Jawaban']}")
                break
        print("=" * 100)
        print()

        prediction_results.append((predicted_label))
        prediction_time.append(end_time - start_time)
    
    return prediction_results, prediction_time


In [20]:
model_results = {}
from copy import deepcopy
for model_name, model_class, model_nickname in models:
    print(f"Deploying ./results/{model_name}_fine-tuned_mental-health")
    prediction_results, prediction_time = deploy_model(model_name, model_class)
    print("*" * 100)
    print()

    test_sets['Predicted Tag'] = prediction_results
    test_sets['Prediction Time'] = prediction_time

    model_results[model_nickname] = deepcopy(test_sets)

Deploying ./results/bert-base-uncased_fine-tuned_mental-health
Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   23,440,896
│    │    └─Embedding: 3-2                                   393,216
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dr

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Layer (type:depth-idx)                                            Param #
MobileBertForSequenceClassification                               --
├─MobileBertModel: 1-1                                            --
│    └─MobileBertEmbeddings: 2-1                                  --
│    │    └─Embedding: 3-1                                        3,906,816
│    │    └─Embedding: 3-2                                        262,144
│    │    └─Embedding: 3-3                                        1,024
│    │    └─Linear: 3-4                                           197,120
│    │    └─NoNorm: 3-5                                           1,024
│    │    └─Dropout: 3-6                                          --
│    └─MobileBertEncoder: 2-2                                     --
│    │    └─ModuleList: 3-7                                       20,213,760
│    └─MobileBertPooler: 2-3                                      --
├─Dropout: 1-2                                                    -

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   24,576,000
│    │    └─Embedding: 3-2                                   393,216
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

# Evaluasi

In [21]:
def save_to_excel(data, file_name, sheet_name):
    # Try to open the existing Excel file and replace the sheet, or create a new file if it doesn't exist
    try:
        # Load the existing Excel file
        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # Write to the specified sheet, replacing if it exists
            data.to_excel(writer, sheet_name=sheet_name, index=False)
    except FileNotFoundError:
        # Create a new Excel file if it doesn't exist
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            data.to_excel(writer, sheet_name=sheet_name, index=False)

In [22]:
RESULT_FILE = 'results/mental_health_prediction_results.xlsx'
model_nicknames = []
model_training_time = []
model_accuracy = []
model_average_prediction_time = []
for model_nickname in model_results:
    print(f"Results for {model_nickname}")
    # print(model_results[model_nickname].columns)
    print(model_results[model_nickname].head())
    print()

    training_duration = training_durations[model_nickname]
    correct_predictions = (model_results[model_nickname]['Tag'] == model_results[model_nickname]['Predicted Tag']).sum()
    total_predictions = len(model_results[model_nickname])
    accuracy = correct_predictions / total_predictions
    average_prediction_time = model_results[model_nickname]['Prediction Time'].mean()

    print(f"Training duration: {training_duration} seconds")
    print(f"Accuracy: {accuracy}")
    print(f"Average prediction time: {average_prediction_time} seconds")
    print()

    model_nicknames.append(model_nickname)
    model_training_time.append(training_duration)
    model_accuracy.append(accuracy)
    model_average_prediction_time.append(average_prediction_time)

    # Sort by tag name
    model_results[model_nickname] = model_results[model_nickname].sort_values(by=['Tag'])
    # Generate the classification report as a dictionary
    report_dict = classification_report(model_results[model_nickname]['Tag'], 
                                        model_results[model_nickname]['Predicted Tag'], 
                                        zero_division=0, output_dict=True)
    # Convert the report dictionary into a DataFrame
    df_report = pd.DataFrame(report_dict).transpose()
    # Sort the DataFrame by precision (descending order)
    df_report_sorted = df_report.sort_values(by='precision', ascending=False)
    # Print the sorted DataFrame
    print(df_report_sorted)
    print(classification_report(model_results[model_nickname]['Tag'], model_results[model_nickname]['Predicted Tag'], zero_division=0))
    print()

    print(np.unique(model_results[model_nickname]['Tag']))
    print(confusion_matrix(model_results[model_nickname]['Tag'], model_results[model_nickname]['Predicted Tag']))
    print("*" * 100)
    print()

    save_to_excel(model_results[model_nickname], RESULT_FILE, model_nickname)

comparison_results = pd.DataFrame({
    'Model': model_nicknames,
    'Training Time': model_training_time,
    'Accuracy': model_accuracy,
    'Average Prediction Time': model_average_prediction_time
})

comparison_results = comparison_results.sort_values(by=['Accuracy'], ascending=False)
print(comparison_results)

save_to_excel(comparison_results, RESULT_FILE, 'Comparison Results')

Results for bert-base-uncased
                                            Pertanyaan    Tag Predicted Tag  \
142  Bagaimana fokus pada kekuatan dan pencapaian d...  tag15         tag19   
6    Bagaimana cara mengatasi perasaan tertekan kar...   tag1          tag4   
97   Apa yang terjadi di otak ketika saya tidak tid...  tag10          tag6   
60   Apa saja gejala yang menunjukkan bahwa kurang ...   tag7          tag6   
112  Bagaimana cara mendengarkan teman yang sedang ...  tag12         tag15   

     Prediction Time  
142         0.038538  
6           0.036033  
97          0.034031  
60          0.034537  
112         0.032029  

Training duration: 36.680134534835815 seconds
Accuracy: 0.42
Average prediction time: 0.008240313529968261 seconds

              precision  recall  f1-score  support
tag14          1.000000    0.40  0.571429    10.00
tag12          0.875000    0.70  0.777778    10.00
tag13          0.875000    0.70  0.777778    10.00
tag22          0.800000    0.40  0.5